In [31]:
from selenium import webdriver

In [36]:
#파일 경로
save_path = 'c:/cr/app'
driver_path = 'c:/cd/chromedriver'

# 파일형식
save_type = 'csv'

#헤더 설정
user_agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) " +\
"AppleWebKit/537.36 (KHTML, like Gecko) " + \
"Chrome/37.0.2062.94 Safari/537.36"
headers = {"User-Agent": user_agent}

#접속 URL
#게임: 최고매출
url = 'https://play.google.com/store/apps/category/GAME/collection/topgrossing'
#찾고싶은 앱의 갯수(ex.상위 10)
app_num = 10

In [32]:
browser = webdriver.Chrome(driver_path)
browser.implicitly_wait(3)
browser.get(url)
#body-content > div.outer-container > div > div.main-content > div > div > div > div.id-card-list.card-list.two-cards > div:nth-child(1)
#body-content > div.outer-container > div > div.main-content > div > div > div > div.id-card-list.card-list.two-cards > div:nth-child(2)

# 1. 접속
## - url 접속
## - 앱 클릭

# 2. 크롤링
## 리뷰 모두 보기 클릭
## 원하는 정도까지 스크롤 다운
## 리뷰 크롤링 (별점, 고객리뷰)


In [34]:
class ARcrawler:
    def __init__(self, url, app_num):
        self.url = url
        self.app_num = app_num
    
    

In [42]:
def click_app(browser, index):
    app = '//*[@id="body-content"]/div/div/div[1]/div/div/div/div[2]/div[{}]'.format(index)
    browser.find_element_by_xpath(app).click()